In [1]:
pip install transformers

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 44.5 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 895 kB 45.0 MB/s 
     |████████████████████████████████| 3.3 MB 42.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
pip install streamlit

In [2]:
!pip install -q sentencepiece

In [3]:
%%writefile app.py
from transformers import pipeline,MBartForConditionalGeneration,MBart50Tokenizer
import streamlit as st
import pandas as pd
import json # to dump & load

def side_menu(key,plot=False):
  st.write("""
   # NLP Pipelines

  This app can do anything with text just select  and apply.
  This is built using Transformers pipeline.

  ```python
  #Features of  Transformer Pipelines.
  1. Sentiment Analysis - classify a sentence positive or negative.
  2. Summary Generation -generate summary from paragraph.
  3. Named Entity Recognition - identify entities.
  4. Question Answering - can answer your question instantly.
  5. Text Generation - generate text
  6. Fill-mask - it is like fill in the blank.
  7. Translation - translate your content .
  ```
  """)
with st.sidebar:
 
  side_menu("sidebar")
def main():
  st.title("NLP App") #title 
  menu=["--Select--","Sentiment Analysis","Summarizer","Named Entity Recognition","Fill mask","Text Generation","Translation","Question-Answer","Zero-shot-Classifier"] # create menu
  choice=st.sidebar.selectbox("Choose any service !",menu) # sidebar-menu
  if choice=="--Select--": # selection 1
    st.write("""NLP is a field of linguistics and machine learning focused on understanding everything related to human language. 
    The aim of NLP tasks is not only to understand single words individually, but to be able to understand the context of those words.""")
    st.write("""The following is a list of common NLP tasks, with some examples of each:

Classifying whole sentences: Getting the sentiment of a review, detecting if an email is spam, determining if a sentence is grammatically correct or whether two sentences are logically related or not

Classifying each word in a sentence: Identifying the grammatical components of a sentence (noun, verb, adjective), or the named entities (person, location, organization)

Generating text content: Completing a prompt with auto-generated text, filling in the blanks in a text with masked words

Extracting an answer from a text: Given a question and a context, extracting the answer to the question based on the information provided in the context

Generating a new sentence from an input text: Translating a text into another language, summarizing a text""")
    
#*********************Summary****************************************#    
  elif choice=="Summarizer": # choice 1
    st.subheader("Text Summary") #subheading
    with st.form(key='nlpForm'):
      raw_text=st.text_area(" Enter the Context") # display box for entering text
      num_wrd=st.number_input("Number of words in Summary") # display box for entering number
      sub_btn=st.form_submit_button(label="Summarize")
      if sub_btn:
        num_wrd=int(num_wrd)
        summarizer=pipeline('summarization') # pipelines Summarziation
        sumry=summarizer(raw_text,min_length=num_wrd,max_length=100) # making summary with max_length i.e 100
        j1=json.dumps(sumry[0])
        j2=json.loads(j1)
        result=j2['summary_text']
      # Here we are joining the sentence with fullstop-. then make the first letter of sentence as Capital
        result='.'.join(list(map(lambda x: x.strip().capitalize(),result.split('.'))))
        st.write(f'Brief :\n {result}')
  
      

#*********************Sentiment Analysis**********************************#      
  elif choice=="Sentiment Analysis": # choice2
    st.subheader("Sentiment Analysis")
    sent=pipeline('sentiment-analysis')
    raw_text=st.text_area("Enter Text")
    btn=st.button('Analyze')
    if btn:
      result=sent(raw_text)[0]
      sentiments=result['label']
      if sentiments=="POSITIVE":
        st.markdown("Positive :smiley: ") 
      elif sentiments=="NEGATIVE":
        st.markdown("Negative :triumph:")
      elif sentiments=="NEUTRAL":
        st.markdown("Neutral :expressionless:")
#*******************Question Answering*****************************#
  elif choice=="Question-Answer":
    st.subheader("Question Answering")
    st.write("Enter the Context")
    raw_text=st.text_area("Result","Enter Context here")
    ques=st.text_area("Question","Enter Question here")
    if raw_text!="Enter Context here" and ques!="Enter Question here":
      qa=pipeline('question-answering')
      result=qa(question=ques,context=raw_text)
      r1=json.dumps(result)
      r2=json.loads(r1)
      final_result=r2['answer']
      final_result='.'.join(map(lambda y: y.strip().capitalize(),final_result.split('.')))
      st.write(f' Answer : \n {final_result}')
#***************Text Generation*************************************#
  elif choice=="Text Generation":
    st.subheader("Text Geneartion from a piece of sentence")
    raw_text=st.text_area("Enter Context Here")
    bty=st.button('Generate')
    if bty :
      text_gen=pipeline('text-generation')
      result=text_gen(raw_text,num_return_sequences=10)
      t1=json.dumps(result[0])
      t2=json.loads(t1)
      text_final=t2['generated_text']
      text_final='.'.join(map(lambda y: y.strip().capitalize(),text_final.split('.')))
      st.write(f" Generated text is :\n  {text_final}")

#*********************Named Entity Recognition******************#
  elif choice=="Named Entity Recognition":
    st.subheader("Named Entity Recognition")
    raw_text=st.text_area("Enter Context Here")
    btg=st.button("Find Entites")
    if btg:
      nlp=pipeline('ner', grouped_entities=True)
      res=nlp(raw_text)
      for i in res:
        sts=[]
        x=i['word']
        sts.append(x)
        st.write(f"```Entity``` are :\n {sts}")
#************Fill-Mask ***********************************#     
  elif choice=="Fill mask":
    st.subheader("Fill mask ")
    st.write("Replace ```-``` with ```[MASK]```")
    st.write("Example: What is ```__``` in machine learning.:thought_balloon:")
    raw_text=st.text_area("Enter Sentence")
    btg=st.button(' Find Mask')
    if btg:
      fm=pipeline('fill-mask',model="bert-base-uncased")
      res=fm(raw_text)
      i=1
      for j in res:
        st.write("```Trial```",i,":",j["sequence"],"``Score``:",str(j["score"]))
        i=i+1
#**************Translator***************************#
  elif choice=="Translation":
    model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt") # download model
    tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt") # download tokenizer
    st.subheader("A tool like Google Translator.")
    st.write("Translate ```Hindi```  to ```English``` ")
    raw_text=st.text_area("Enter the Text")
    btn=st.button("Translate")
    if btn:
      tokenizer.src_lang = "hi_IN"
      encoded_hi = tokenizer(raw_text, return_tensors="pt")
      generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
      out=tokenizer.batch_decode(generated_tokens, skip_special_tokens=True) #decode language
      st.write('',str(out).strip('][\''))
#***********************Zero-Shot-Classifier********************#

  elif choice=="Zero-shot-Classifier":
    st.subheader("Zero Shot classifier")
    st.write("Predict ```category``` that are belong to ```context```")
    raw_text=st.text_area("Enter Context")
    cat=st.text_area("Enter Categories",value='',height=None,max_chars=None,key=None)
    but=st.button("Predict")
    if but:
      classifier = pipeline("zero-shot-classification")
      res=classifier(raw_text,cat)
      l1=res['labels']
      l2=res['scores']
      for i,j in zip(l1,l2):
        st.write(f'```Labels``` are : {i}',  f'```Scores``` are : {j}')
     

if __name__=="__main__":
  main() # Calling
    


Writing app.py


In [4]:
!streamlit run app.py --server.maxUploadSize=1028 & npx localtunnel --port 8501

npx: installed 22 in 4.604s
your url is: https://nasty-dragon-56.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.188.179.0:8501

  Stopping...
^C
